In [38]:
import numpy as np

def demographic_parity_difference(predictions, sensitive_attribute):

    # convert preds and sensitive attribute to arrays
    predictions = np.array(predictions)
    sensitive_attribute = np.array(sensitive_attribute)
    
    # get labels + sensitive attribute vals
    unique_labels = np.unique(predictions)
    unique_sensitive_attribute = np.unique(sensitive_attribute)

    # print(unique_labels)
    # print(sensitive_attribute)
    
    # store dpd
    dp_diff = np.zeros(len(unique_labels))
    privileged_gender = np.empty(len(unique_labels), dtype=object)

    
    # compute dpd for each label
    for i, label in enumerate(unique_labels):
        # find indices where pred is equal to current label
        label_indices = np.where(predictions == label)[0]
        
        # compute proportions of positive outcomes for each group
        group_proportions = []
        for a in unique_sensitive_attribute:
            # find indices where sensitive attribute is equal to current value
            group_indices = np.where(sensitive_attribute == a)[0]
            
            # compute proportion of positive outcomes for this group
            positive_proportion = np.mean(predictions[group_indices] == label)
            group_proportions.append(positive_proportion)
        
        # compute dpd for this label
        dp_diff[i] = max(group_proportions) - min(group_proportions)

        # ID privileged gender for this label
        privileged_index = np.argmax(group_proportions)
        privileged_gender[i] = unique_sensitive_attribute[privileged_index]
    
    return dp_diff, privileged_gender


In [39]:
import numpy as np

def equal_opportunity_difference(predictions, true_labels, sensitive_attribute):

    # convert preds and sensitive attribute to arrays
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    sensitive_attribute = np.array(sensitive_attribute)
    
    # get labels + sensitive attribute vals
    unique_labels = np.unique(predictions)
    unique_sensitive_attribute = np.unique(sensitive_attribute)
    
    # store eod
    eo_diff = np.zeros(len(unique_labels))
    privileged_gender = np.empty(len(unique_labels), dtype=object)
    
    # compute eod for each label
    for i, label in enumerate(unique_labels):
        # find indices where pred is equal to current label
        label_indices = np.where(predictions == label)[0]
        
        # compute TPR for each group
        tpr_group = []
        for a in unique_sensitive_attribute:
            # find indices where sensitive attribute is equal to current value
            group_indices = np.where(sensitive_attribute == a)[0]
            
            # compute TPR for this group
            true_positives = np.sum((predictions[group_indices] == label) & (true_labels[group_indices] == label))
            actual_positives = np.sum(true_labels[group_indices] == label)
            
            if actual_positives > 0:
                true_positive_rate = true_positives / actual_positives
                tpr_group.append(true_positive_rate)
            else:
                tpr_group.append(0.0)  # division by zero case
        
        # compute eod for this label
        eo_diff[i] = max(tpr_group) - min(tpr_group)
        
        # ID privileged gender for this label
        privileged_index = np.argmax(tpr_group)
        privileged_gender[i] = unique_sensitive_attribute[privileged_index]
    
    return eo_diff, privileged_gender


In [40]:
import numpy as np

def average_odds_difference(predictions, true_labels, sensitive_attribute):

    # convert preds and sensitive attribute to arrays
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    sensitive_attribute = np.array(sensitive_attribute)
    
    # get labels + sensitive attribute vals
    unique_labels = np.unique(predictions)
    unique_sensitive_attribute = np.unique(sensitive_attribute)
    
    # store aod
    aod_diff = np.zeros(len(unique_labels))
    privileged_gender = np.empty(len(unique_labels), dtype=object)
    
    # compute aod for each label
    for i, label in enumerate(unique_labels):
        # find indices where pred is equal to current label
        label_indices = np.where(predictions == label)[0]
        
        # compute FPR and TPR for each group
        fpr_group = []
        tpr_group = []
        for a in unique_sensitive_attribute:
            # find indices where sensitive attribute is equal to current value
            group_indices = np.where(sensitive_attribute == a)[0]
            
            # compute FPR for this group
            false_positives = np.sum((predictions[group_indices] == label) & (true_labels[group_indices] != label))
            actual_negatives = np.sum(true_labels[group_indices] != label)
            if actual_negatives > 0:
                false_positive_rate = false_positives / actual_negatives
                fpr_group.append(false_positive_rate)
            else:
                fpr_group.append(0.0)  # division by zero case
            
            # compute TPR for this group
            true_positives = np.sum((predictions[group_indices] == label) & (true_labels[group_indices] == label))
            actual_positives = np.sum(true_labels[group_indices] == label)
            if actual_positives > 0:
                true_positive_rate = true_positives / actual_positives
                tpr_group.append(true_positive_rate)
            else:
                tpr_group.append(0.0)  # division by zero case
        
        # compute aod for this label
        aod_diff[i] = (max(fpr_group) - min(fpr_group) + max(tpr_group) - min(tpr_group)) / 2
        
        # ID privileged gender for this label
        privileged_index = np.argmax(tpr_group)
        privileged_gender[i] = unique_sensitive_attribute[privileged_index]
    
    return aod_diff, privileged_gender


In [41]:
import numpy as np

def disparate_impact(predictions, sensitive_attribute):

    # convert preds and sensitive attribute to arrays
    predictions = np.array(predictions)
    sensitive_attribute = np.array(sensitive_attribute)
    
    # get labels + sensitive attribute vals
    unique_labels = np.unique(predictions)
    unique_sensitive_attribute = np.unique(sensitive_attribute)
    
    # store di ratio
    di_ratio = np.zeros(len(unique_labels))
    privileged_gender = np.empty(len(unique_labels), dtype=object)
    
    # compute di ratio for each label
    for i, label in enumerate(unique_labels):
        # find indices where pred is equal to current label
        label_indices = np.where(predictions == label)[0]
        
        # compute proportions of positive outcomes for each group
        favorable_proportions = []
        for a in unique_sensitive_attribute:
            # find indices where sensitive attribute is equal to current value
            group_indices = np.where(sensitive_attribute == a)[0]
            
            # compute proportion of positive outcomes for this group
            favorable_proportion = np.mean(predictions[group_indices] == label)
            favorable_proportions.append(favorable_proportion)
        
        # compute DI ratio for this label
        di_ratio[i] = favorable_proportions[1] / favorable_proportions[0] if favorable_proportions[0] != 0 else 0
        
        # ID privileged gender for this label
        privileged_index = np.argmax(favorable_proportions)
        privileged_gender[i] = unique_sensitive_attribute[privileged_index]
    
    return di_ratio, privileged_gender


Data


In [42]:
# preprocess biased dataset

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


data = pd.read_csv('biased.csv')

# drop rows with missing values in text columns
text_columns = ['Name', 'Education', 'Work Experience', 'Skills', 'Awards', 'Job']
data = data.dropna(subset=text_columns)

# extract text data and labels
X_text = data['Name'] + ' ' + data['Education'] + ' ' + data['Work Experience'] + ' ' + data['Skills'] + ' ' + data['Awards']
y = data['Fit']
gender = data['Gender']

# vectorize text data with TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_text_vec = vectorizer.fit_transform(X_text)

# split dataset
X_train, X_test, y_train, y_test, gender_train, gender_test = train_test_split(X_text_vec, y, gender, test_size=0.2, random_state=42)



SVM


In [43]:
# train SVM

svm_classifier = SVC(kernel='linear', C=1.0, random_state=42) 
svm_classifier.fit(X_train, y_train)

# predict on test set
y_pred = svm_classifier.predict(X_test)

dp_diff = demographic_parity_difference(y_pred, gender_test)
print("Demographic parity difference for each label:", dp_diff)

eo_diff = equal_opportunity_difference(y_pred, y_test, gender_test)
print("Equal opportunity difference for each label:", eo_diff)

ao_diff = average_odds_difference(y_pred, y_test, gender_test)
print("Average odds difference for each label:", ao_diff)

di_ratio = disparate_impact(y_pred, gender_test)
print("Disparate impact for each label:", di_ratio)

# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy:.2f}")

# classification report
print(classification_report(y_test, y_pred))

Demographic parity difference for each label: (array([0.02680653, 0.01759907, 0.10291375, 0.12703963, 0.18554779]), array(['male', 'male', 'female', 'female', 'male'], dtype=object))
Equal opportunity difference for each label: (array([0.31818182, 0.26785714, 0.06868687, 0.58536585, 0.16304348]), array(['male', 'female', 'female', 'female', 'male'], dtype=object))
Average odds difference for each label: (array([0.17000985, 0.13587245, 0.06207184, 0.30686642, 0.13155894]), array(['male', 'female', 'female', 'female', 'male'], dtype=object))
Disparate impact for each label: (array([1.11057692, 1.12226721, 0.66866792, 0.47596154, 4.06153846]), array(['male', 'male', 'female', 'female', 'male'], dtype=object))

Accuracy: 0.59
              precision    recall  f1-score   support

     average       0.46      0.65      0.54        48
         bad       0.78      0.70      0.74        44
        good       0.71      0.62      0.66        78
    very bad       0.51      0.57      0.54        

Log Reg

In [44]:
# train the Logistic Regression classifier

logistic_classifier = LogisticRegression(max_iter=1000, random_state=42)  
logistic_classifier.fit(X_train, y_train)

# predict on test set
y_pred = logistic_classifier.predict(X_test)

dp_diff = demographic_parity_difference(y_pred, gender_test)
print("Demographic parity difference for each label:", dp_diff)

eo_diff = equal_opportunity_difference(y_pred, y_test, gender_test)
print("Equal opportunity difference for each label:", eo_diff)

ao_diff = average_odds_difference(y_pred, y_test, gender_test)
print("Average odds difference for each label:", ao_diff)

di_ratio = disparate_impact(y_pred, gender_test)
print("Disparate impact for each label:", di_ratio)


# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# classification report
print(classification_report(y_test, y_pred))


Demographic parity difference for each label: (array([0.05757576, 0.01759907, 0.11818182, 0.12692308, 0.16993007]), array(['male', 'male', 'female', 'female', 'male'], dtype=object))
Equal opportunity difference for each label: (array([0.3951049 , 0.26785714, 0.07676768, 0.58536585, 0.39130435]), array(['male', 'female', 'female', 'female', 'male'], dtype=object))
Average odds difference for each label: (array([0.19938683, 0.13587245, 0.0776065 , 0.30524788, 0.23173848]), array(['male', 'female', 'female', 'female', 'male'], dtype=object))
Disparate impact for each label: (array([1.2375    , 1.12226721, 0.62857143, 0.49230769, 4.73846154]), array(['male', 'male', 'female', 'female', 'male'], dtype=object))
Accuracy: 0.56
              precision    recall  f1-score   support

     average       0.41      0.60      0.49        48
         bad       0.78      0.70      0.74        44
        good       0.68      0.59      0.63        78
    very bad       0.49      0.57      0.53        4

Random Forest

In [45]:
# train Random Forest

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  
rf_classifier.fit(X_train, y_train)

# predict on test set
y_pred = rf_classifier.predict(X_test)

dp_diff = demographic_parity_difference(y_pred, gender_test)
print("Demographic parity difference for each label:", dp_diff)

eo_diff = equal_opportunity_difference(y_pred, y_test, gender_test)
print("Equal opportunity difference for each label:", eo_diff)

ao_diff = average_odds_difference(y_pred, y_test, gender_test)
print("Average odds difference for each label:", ao_diff)

di_ratio = disparate_impact(y_pred, gender_test)
print("Disparate impact for each label:", di_ratio)

# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# classification report
print(classification_report(y_test, y_pred))

Demographic parity difference for each label: (array([0.01060606, 0.0488345 , 0.09452214, 0.08146853, 0.11655012]), array(['male', 'male', 'female', 'female', 'male'], dtype=object))
Equal opportunity difference for each label: (array([0.26223776, 0.08035714, 0.01010101, 0.53658537, 0.04347826]), array(['male', 'female', 'female', 'female', 'female'], dtype=object))
Average odds difference for each label: (array([0.1409896 , 0.05015153, 0.01950719, 0.30283566, 0.02917961]), array(['male', 'female', 'female', 'female', 'female'], dtype=object))
Disparate impact for each label: (array([1.056     , 1.28026756, 0.73453355, 0.6017094 , 2.53846154]), array(['male', 'male', 'female', 'female', 'male'], dtype=object))
Accuracy: 0.73
              precision    recall  f1-score   support

     average       0.76      0.81      0.79        48
         bad       0.75      0.89      0.81        44
        good       0.85      0.88      0.87        78
    very bad       0.51      0.52      0.52     

Grad Boost

In [46]:
# train Gradient Boosting classifier

gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_classifier.fit(X_train, y_train)

# predict on test set
y_pred = gb_classifier.predict(X_test)

dp_diff = demographic_parity_difference(y_pred, gender_test)
print("Demographic parity difference for each label:", dp_diff)

eo_diff = equal_opportunity_difference(y_pred, y_test, gender_test)
print("Equal opportunity difference for each label:", eo_diff)

ao_diff = average_odds_difference(y_pred, y_test, gender_test)
print("Average odds difference for each label:", ao_diff)

di_ratio = disparate_impact(y_pred, gender_test)
print("Disparate impact for each label:", di_ratio)

# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# classification report
print(classification_report(y_test, y_pred))


Demographic parity difference for each label: (array([0.05652681, 0.03310023, 0.10291375, 0.09615385, 0.10944056]), array(['male', 'male', 'female', 'female', 'male'], dtype=object))
Equal opportunity difference for each label: (array([0.13286713, 0.15178571, 0.05050505, 0.58536585, 0.04347826]), array(['male', 'female', 'female', 'female', 'male'], dtype=object))
Average odds difference for each label: (array([0.09849157, 0.08764066, 0.04427137, 0.32075176, 0.04871086]), array(['male', 'female', 'female', 'female', 'male'], dtype=object))
Disparate impact for each label: (array([1.32441472, 1.21846154, 0.66866792, 0.61538462, 1.96307692]), array(['male', 'male', 'female', 'female', 'male'], dtype=object))
Accuracy: 0.71
              precision    recall  f1-score   support

     average       0.72      0.79      0.75        48
         bad       0.84      0.84      0.84        44
        good       0.87      0.76      0.81        78
    very bad       0.45      0.57      0.51        4